# Market Basket Analysis in Python ①
> [Market Basket Analysis in Python ①Introduction to Market Basket Analysis](https://learn.datacamp.com/courses/market-basket-analysis-in-python) まとめ, via Datacamp

- toc: true
- badges: false
- comments: true
- author: Jay Sung
- categories: [ ___  D. MARKETING ____________ > Datacamp Class]
- image: images/basket.png

- - -

## バスケット分析とは？

バスケット分析によって以下のようなことができる。  

### 1. 併売分析  
一回の買い物でよく一緒に買われるものは何か？

### 2. 併売分析に基づいて有用な推奨事項の提供
e.g.   
・棚の近くに置くべき商品カテゴリー  
・棚の遠くに置くべき商品カテゴリー など


- - -

## バスケット分析の使用例

e.g.  
・Netflixスタイルの推奨エンジン  
・eコマースにおける商品推奨の改善  
・小売店でのクロスセル（Cross-sell$^{[*1]}$）製品  
・在庫管理の改善  
・アップセル（Up-sell$^{[*2]}$）するアイテム選択　など

<br>  

<span style="font-size: 85%"> 
    $^{[*1]}$ Cross-sell :  顧客が購買する、あるいはすでに利用している製品・サービスに関連するものを販売していくこと  
    $^{[*2]}$ Up-sell : 顧客が検討しているものよりランクの高い製品・サービスの販売をすること

</span> 

- - -

## バスケット分析の方法

 **Association Rule** によって構成される

・{A} $\rightarrow$ {B}

「Aを買うと、Bを買う」という意味
- - -

## (実装) クロルセル製品選択

小売店ではガムをコーヒーかシリアルかパンかどれかとクロスセルしようとしている。  

・{coffee} $\rightarrow$ {gum}  
・{cereal} $\rightarrow$ {gum}  
・{bread} $\rightarrow$ {gum}  のうち、最も関係性が強いものを選びたい。  

<br>  

`transactions`は ['coffee','gum'] & ['cereal','gum'] & ['bread','gum'] の３つのペアからなっている。

In [1]:
#
# import & read data

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

transactions = pd.read_csv('./dataset/transactions.csv', header=None)

transactions

,0,1
0,bread,gum
1,bread,gum
2,cereal,gum
3,coffee,gum
4,bread,gum
...,...,...
80,bread,gum
81,coffee,gum
82,bread,gum
83,coffee,gum


各ペアの数をカウントする

In [2]:
#
# Count the number of transactions with coffee and gum
coffee = transactions[(transactions[0] == 'coffee') & (transactions[1] == 'gum')]

# Count the number of transactions with cereal and gum
cereal = transactions[(transactions[0] == 'cereal') & (transactions[1] == 'gum')]

# Count the number of transactions with bread and gum
bread = transactions[(transactions[0] == 'bread') & (transactions[1] == 'gum')]

# Print the counts for each transaction.
print('coffee:', len(coffee))
print('cereal:', len(cereal))
print('bread:', len(bread))

coffee: 40
cereal: 25
bread: 20


- - -

## Association Rulesの全ての候補を確認する

### データの準備

小売店の併売データを読み込む

In [3]:
#
# Load transactions from pandas
groceries = pd.read_csv('./dataset/small_grocery_store.csv')

# Split transaction strings into lists
transactions = groceries['Transaction'].apply(lambda t: t.split(','))

# Convert DataFrame column into list of strings
transactions = list(transactions)

# Print the list of transactions
print(transactions)

[['milk', 'bread', 'biscuit'], ['bread', 'milk', 'biscuit', 'cereal'], ['bread', 'tea'], ['jam', 'bread', 'milk'], ['tea', 'biscuit'], ['bread', 'tea'], ['tea', 'cereal'], ['bread', 'tea', 'biscuit'], ['jam', 'bread', 'tea'], ['bread', 'milk'], ['coffee', 'orange', 'biscuit', 'cereal'], ['coffee', 'orange', 'biscuit', 'cereal'], ['coffee', 'sugar'], ['bread', 'coffee', 'orange'], ['bread', 'sugar', 'biscuit'], ['coffee', 'sugar', 'cereal'], ['bread', 'sugar', 'biscuit'], ['bread', 'coffee', 'sugar'], ['bread', 'coffee', 'sugar'], ['tea', 'milk', 'coffee', 'cereal']]


### Association rulesの生成

９つの食料品のリストを抽出し、すべての可能な単独原因単独結果の組み合わせ（Association rules）をitertoolsを使って生成する

In [4]:
#
# Import permutations from the itertools module
from itertools import permutations

# Define the set of groceries
flattened = [i for t in transactions for i in t]
groceries = list(set(flattened))

# Generate all possible rules
rules = list(permutations(groceries, 2))

# Print the set of rules
print(rules)

# Print the number of rules
print('\n')
print(len(rules),"通り")

[('jam', 'cereal'), ('jam', 'sugar'), ('jam', 'tea'), ('jam', 'orange'), ('jam', 'bread'), ('jam', 'milk'), ('jam', 'coffee'), ('jam', 'biscuit'), ('cereal', 'jam'), ('cereal', 'sugar'), ('cereal', 'tea'), ('cereal', 'orange'), ('cereal', 'bread'), ('cereal', 'milk'), ('cereal', 'coffee'), ('cereal', 'biscuit'), ('sugar', 'jam'), ('sugar', 'cereal'), ('sugar', 'tea'), ('sugar', 'orange'), ('sugar', 'bread'), ('sugar', 'milk'), ('sugar', 'coffee'), ('sugar', 'biscuit'), ('tea', 'jam'), ('tea', 'cereal'), ('tea', 'sugar'), ('tea', 'orange'), ('tea', 'bread'), ('tea', 'milk'), ('tea', 'coffee'), ('tea', 'biscuit'), ('orange', 'jam'), ('orange', 'cereal'), ('orange', 'sugar'), ('orange', 'tea'), ('orange', 'bread'), ('orange', 'milk'), ('orange', 'coffee'), ('orange', 'biscuit'), ('bread', 'jam'), ('bread', 'cereal'), ('bread', 'sugar'), ('bread', 'tea'), ('bread', 'orange'), ('bread', 'milk'), ('bread', 'coffee'), ('bread', 'biscuit'), ('milk', 'jam'), ('milk', 'cereal'), ('milk', 'sugar'

- - -

## Metric & Pruning

全ての可能な組み合わせを候補として置くのは、かなり時間とコストがかかる。  
そのため、有用な規則だけに絞りたい。  → **Metric & Pruning Process**  



①**Metric** : Association rulesにスコアを割り当て

②**Pruning** : 特定の基準に従い、基準に満たさないRulesを放棄する

### One-hot encoding transaction data

In [6]:
#
# Import the transaction encoder function from mlxtend
from mlxtend.preprocessing import TransactionEncoder

# Instantiate transaction encoder and identify unique items
encoder = TransactionEncoder().fit(transactions)

# One-hot encode transactions
onehot = encoder.transform(transactions)

# Convert one-hot encoded data to DataFrame
onehot = pd.DataFrame(onehot, columns = encoder.columns_)

# Print the one-hot encoded transaction dataset
print(onehot)

    biscuit  bread  cereal  coffee    jam   milk  orange  sugar    tea
0      True   True   False   False  False   True   False  False  False
1      True   True    True   False  False   True   False  False  False
2     False   True   False   False  False  False   False  False   True
3     False   True   False   False   True   True   False  False  False
4      True  False   False   False  False  False   False  False   True
5     False   True   False   False  False  False   False  False   True
6     False  False    True   False  False  False   False  False   True
7      True   True   False   False  False  False   False  False   True
8     False   True   False   False   True  False   False  False   True
9     False   True   False   False  False   True   False  False  False
10     True  False    True    True  False  False    True  False  False
11     True  False    True    True  False  False    True  False  False
12    False  False   False    True  False  False   False   True  False
13    

In [7]:
#
# Compute the support
support = onehot.mean()

# Print the support
print(support)

biscuit    0.40
bread      0.65
cereal     0.30
coffee     0.40
jam        0.10
milk       0.25
orange     0.15
sugar      0.30
tea        0.35
dtype: float64


In [9]:
#
# Add a jam+bread column to the DataFrame onehot
onehot['jam+bread'] = np.logical_and(onehot['jam'], onehot['bread'])

# Compute the support
support = onehot.mean()

# Print the support values
print(support)

biscuit      0.40
bread        0.65
cereal       0.30
coffee       0.40
jam          0.10
milk         0.25
orange       0.15
sugar        0.30
tea          0.35
jam+bread    0.10
dtype: float64


<br>  


**Additional Reference**  
JMR生活総合研究所. アップセリング（Up-selling）、クロスセリング（Cross-selling）. *マーケティング用語集*. https://www.jmrlsi.co.jp/knowledge/yougo/my06/my0618.html



- - -

**このシリーズの別のポストを見るには**

　　　[[Next >>]](https://jaysung00.github.io/jays_blog/datacamp/2021/02/09/Basket-analysis-2.html)
   

